In [1]:
import pandas as pd
import numpy as np
import os,random

In [2]:
seed=42
def seed_everything(seed=0):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(seed)

In [3]:
train=pd.read_csv('/content/drive/MyDrive/ulianov/train_exif.csv')
test=pd.read_csv('/content/drive/MyDrive/ulianov/test_exif.csv')

In [4]:
train['width']=train['x_max']-train['x_min']
train['height']=train['y_max']-train['y_min']

test=test.drop(columns=['width','height'])

test['width']=test['x_max']-test['x_min']
test['height']=test['y_max']-test['y_min']

In [5]:
drops=['image_name','class']

train=train.drop(columns=drops)



In [6]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
pd.set_option('display.max_columns', 500)
from catboost import CatBoostRegressor

In [8]:
#print(train['OffsetTimeDigitized'].value_counts())

In [9]:


def prepare(x):
    x = x.split(' ')
    date = x[0].split(':')
    time = x[1].split(':')
    year,month,day = map(int, date)
    timestamp = pd.Timestamp(f'{day}-{month}-{year}')
    hour = int(time[0])
    dayofyear = timestamp.dayofyear
    dayofweek = timestamp.dayofweek
    if dayofweek >= 5:
        days_to_weekend = 0
    else:
        days_to_weekend = min(dayofweek + 1, abs(5 - dayofweek))
    return year, month, day, hour, dayofyear, dayofweek, dayofweek>=5, (month-1) // 3, days_to_weekend

add_columns = ['year', 'month', 'day', 'hour', 'dayofyear', 'dayofweek', 'is_weekend', \
               'season', 'days_to_weekend']
train[add_columns] = list(train['DateTime'].apply(prepare))
for col in add_columns:
    train[col] = train[col].astype(np.int32)
  
test[add_columns] = list(test['DateTime'].apply(prepare))
for col in add_columns:
    test[col] = test[col].astype(np.int32)
  
date_drops=['DateTimeOriginal','DateTimeDigitized','DateTime']
train=train.drop(columns=date_drops)
test=test.drop(columns=date_drops)

In [10]:
for i in train.columns:#                     
  print(train[i].value_counts()/len(train))
  print()

full_disbalanced=['ResolutionUnit','Make','XResolution','YResolution','ExifVersion','Flash','ColorSpace','ExifImageWidth','OffsetTime','OffsetTimeOriginal','OffsetTimeDigitized','ExifImageHeight','SensingMethod','SceneType','ExposureProgram','ExposureMode','WhiteBalance','LensMake','1','2','3','4','5','12','16','23','21_gps','41_gps','year','month','is_weekend','season']
#full_disbalanced=['ResolutionUnit','XResolution','YResolution','Flash','ColorSpace','ExifImageWidth','ExifImageHeight','SensingMethod','ExposureProgram','ExposureMode','WhiteBalance','2','4','21_gps','41_gps','year','month','is_weekend','season','1_N','3_E','12_K','16_T','23_T','Make_Apple',"ExifVersion_b'0232'",'OffsetTime_+04:00','OffsetTimeOriginal_+04:00','OffsetTimeDigitized_+04:00',"SceneType_b'\x01'",'LensMake_Apple',"5_b'\x00'"]
partially_disbalanced=['ApertureValue','Orientation','ExposureBiasValue','MeteringMode','FNumber','CompositeImage','13','lens1','lens2','lens3']

2.10    0.009434
4.42    0.007547
4.41    0.007547
3.24    0.007547
3.52    0.007547
          ...   
6.00    0.001887
4.17    0.001887
7.55    0.001887
5.09    0.001887
5.20    0.001887
Name: distance, Length: 348, dtype: float64

219.604050    0.001887
103.592194    0.001887
149.841110    0.001887
255.470154    0.001887
177.300095    0.001887
                ...   
194.177078    0.001887
267.287781    0.001887
131.098465    0.001887
140.876999    0.001887
181.400436    0.001887
Name: x_min, Length: 524, dtype: float64

402.193756    0.001887
204.923050    0.001887
314.474548    0.001887
702.439575    0.001887
412.821014    0.001887
                ...   
236.720749    0.001887
355.527618    0.001887
231.912506    0.001887
260.147736    0.001887
356.746887    0.001887
Name: y_min, Length: 524, dtype: float64

425.297180    0.001887
238.980759    0.001887
352.669220    0.001887
695.618835    0.001887
440.045044    0.001887
                ...   
406.994904    0.001887
594.195923    0.0

In [11]:
print(list(train.columns))

['distance', 'x_min', 'y_min', 'x_max', 'y_max', 'conf', 'ResolutionUnit', 'ExifOffset', 'Make', 'Model', 'Software', 'Orientation', 'XResolution', 'YResolution', 'HostComputer', 'ExifVersion', 'ShutterSpeedValue', 'ApertureValue', 'BrightnessValue', 'ExposureBiasValue', 'MeteringMode', 'Flash', 'FocalLength', 'ColorSpace', 'ExifImageWidth', 'DigitalZoomRatio', 'FocalLengthIn35mmFilm', 'OffsetTime', 'OffsetTimeOriginal', 'OffsetTimeDigitized', 'SubsecTimeOriginal', 'SubsecTimeDigitized', 'ExifImageHeight', 'SensingMethod', 'ExposureTime', 'FNumber', 'SceneType', 'ExposureProgram', 'ISOSpeedRatings', 'ExposureMode', 'WhiteBalance', 'LensMake', 'LensModel', 'CompositeImage', '1', '2', '3', '4', '5', '6', '12', '13', '16', '17', '23', '24', '31', '21_gps', '22_gps', '23_gps', '41_gps', '42_gps', '43_gps', 'lens1', 'lens2', 'lens3', 'subjcloc1', 'subjcloc2', 'subjcloc3', 'subjcloc4', 'width', 'height', 'year', 'month', 'day', 'hour', 'dayofyear', 'dayofweek', 'is_weekend', 'season', 'days_

In [12]:
#print(train)

In [13]:
train=train.drop(columns=full_disbalanced)
test=test.drop(columns=full_disbalanced)#partially_disbalanced

train=train.drop(columns=partially_disbalanced)
test=test.drop(columns=partially_disbalanced)

In [14]:
print(train['LensModel'].unique())

['iPhone 11 back dual wide camera 1.54mm f/2.4'
 'iPhone 11 back dual wide camera 4.25mm f/1.8'
 'iPhone 12 back dual wide camera 1.55mm f/2.4']


In [15]:
train['camera_mm']=1.54
train.loc[train['LensModel']=='iPhone 11 back dual wide camera 4.25mm f/1.8','camera_mm']=4.25
train.loc[train['LensModel']=='iPhone 12 back dual wide camera 1.55mm f/2.4','camera_mm']=1.55

train['camera_f/']=2.4
train.loc[train['LensModel']=='iPhone 11 back dual wide camera 4.25mm f/1.8','camera_mm']=1.8

test['camera_mm']=1.54
test.loc[test['LensModel']=='iPhone 11 back dual wide camera 4.25mm f/1.8','camera_mm']=4.25
test.loc[test['LensModel']=='iPhone 12 back dual wide camera 1.55mm f/2.4','camera_mm']=1.55

test['camera_f/']=2.4
test.loc[test['LensModel']=='iPhone 11 back dual wide camera 4.25mm f/1.8','camera_mm']=1.8

In [16]:
print(train['camera_mm'].value_counts())

1.54    497
1.55     27
1.80      6
Name: camera_mm, dtype: int64


In [17]:
print(list(train.columns))

['distance', 'x_min', 'y_min', 'x_max', 'y_max', 'conf', 'ExifOffset', 'Model', 'Software', 'HostComputer', 'ShutterSpeedValue', 'BrightnessValue', 'FocalLength', 'DigitalZoomRatio', 'FocalLengthIn35mmFilm', 'SubsecTimeOriginal', 'SubsecTimeDigitized', 'ExposureTime', 'ISOSpeedRatings', 'LensModel', '6', '17', '24', '31', '22_gps', '23_gps', '42_gps', '43_gps', 'subjcloc1', 'subjcloc2', 'subjcloc3', 'subjcloc4', 'width', 'height', 'day', 'hour', 'dayofyear', 'dayofweek', 'days_to_weekend', 'camera_mm', 'camera_f/']


In [18]:
#print(train)
drops=['Make','Model','DateTime','HostComputer','ExifVersion','DateTimeOriginal',
       'DateTimeDigitized','OffsetTime','OffsetTimeOriginal','OffsetTimeDigitized','SceneType',
       'LensMake','5']

#train=pd.get_dummies(train,columns=['LensModel','1','3','12','16','23','Software'])
#test=pd.get_dummies(test,columns=['LensModel','1','3','12','16','23','Software'])

train=pd.get_dummies(train,columns=['LensModel','Software'])
test=pd.get_dummies(test,columns=['LensModel','Software'])

'''train=pd.get_dummies(train,columns=['Make','Model','HostComputer','ExifVersion',
                                    'OffsetTime','OffsetTimeOriginal','OffsetTimeDigitized',
                                    'SceneType','LensMake','5'])
test=pd.get_dummies(test,columns=['Make','Model','HostComputer','ExifVersion',
                                    'OffsetTime','OffsetTimeOriginal','OffsetTimeDigitized',
                                    'SceneType','LensMake','5'])'''

train=pd.get_dummies(train,columns=['Model','HostComputer'])
test=pd.get_dummies(test,columns=['Model','HostComputer'])
#train=train.drop(columns=drops)
#test=test.drop(columns=drops)
print(train)

     distance       x_min        y_min        x_max        y_max      conf  \
0        4.88  219.604050   402.193756   425.297180   455.929901  0.842233   
1        1.54  614.961609  1326.627197  1302.240479  1479.057617  0.864385   
2        3.68  298.759644   461.289703   591.384766   529.101013  0.865617   
3        2.22  501.177734   748.439636   983.731384   856.448853  0.874792   
4        3.73  294.875153   471.319153   591.095154   539.254272  0.868262   
..        ...         ...          ...          ...          ...       ...   
525      3.00  306.027252   650.916504   604.249329   720.184814  0.865165   
526      3.46  269.867737   575.216919   522.212463   635.551270  0.865655   
527      3.91  249.696381   501.747467   478.498352   556.968689  0.857690   
528      5.20  198.687332   382.187531   368.827576   425.248566  0.837795   
529      5.54  181.400436   356.746887   344.161957   398.403748  0.829634   

     ExifOffset  ShutterSpeedValue  BrightnessValue  FocalLengt

In [19]:
X=train.drop('distance',axis=1)
y=train['distance']

cb=CatBoostRegressor(iterations=20000,random_state=seed,task_type='GPU',learning_rate=0.001)
cb.fit(X,y,verbose=1000)

test_vals=test.drop(columns=['image_name','class'])
preds=cb.predict(test_vals)

sub_d={
    'image_name':list(test[['image_name']].values),
    'distance':preds
}

sub=pd.DataFrame(sub_d)
sub['image_name']=list(map(lambda x:x[0],sub['image_name']))
print(sub)
sub.to_csv('/content/drive/MyDrive/ulianov/res.csv',index=False,sep=';')

0:	learn: 1.7557827	total: 32ms	remaining: 10m 39s
1000:	learn: 0.8862432	total: 20s	remaining: 6m 19s
2000:	learn: 0.5604160	total: 29.3s	remaining: 4m 23s
3000:	learn: 0.4424041	total: 38.6s	remaining: 3m 38s
4000:	learn: 0.3925460	total: 47.9s	remaining: 3m 11s
5000:	learn: 0.3651201	total: 57.2s	remaining: 2m 51s
6000:	learn: 0.3457591	total: 1m 6s	remaining: 2m 35s
7000:	learn: 0.3294376	total: 1m 15s	remaining: 2m 21s
8000:	learn: 0.3153125	total: 1m 26s	remaining: 2m 9s
9000:	learn: 0.3033704	total: 1m 35s	remaining: 1m 56s
10000:	learn: 0.2929265	total: 1m 45s	remaining: 1m 45s
11000:	learn: 0.2829148	total: 1m 54s	remaining: 1m 33s
12000:	learn: 0.2739817	total: 2m 3s	remaining: 1m 22s
13000:	learn: 0.2661840	total: 2m 13s	remaining: 1m 11s
14000:	learn: 0.2591253	total: 2m 22s	remaining: 1m 1s
15000:	learn: 0.2529752	total: 2m 32s	remaining: 50.7s
16000:	learn: 0.2473349	total: 2m 41s	remaining: 40.3s
17000:	learn: 0.2423501	total: 2m 50s	remaining: 30.1s
18000:	learn: 0.2375